In [1]:
import requests 
import pandas as pd 
import numpy as np 
import random 
import folium 
from geopy.geocoders import Nominatim 

1.	Convierte la dirección del centro de tu ciudad en longitud y latitud y después busca los restaurantes de comida italiana más cercanos a 3000 metros.


In [3]:
address = 'Puerta Sol, Madrid, España'

geolocator = Nominatim(user_agent="foursquare_agent") #A través de foursquare transforma la dirección en latitud y longitud
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.416865400000006 -3.7043021694877964


2.	Obtén un dataframe con los nombres, categorías, dirección, codigo postal, latitud, longitud e ID de los restaurantes.


In [4]:
search_query = 'Italian'
radius = 3000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=U1LHGA0A24XNVNYN4FQJK3MD1VCBZ0KPP2LO12XQTS3FCN21&client_secret=N5HHZVFVPRVU2A3BMHFXU2UW1ODJ5QIM4PUQM0UJSCFIYRNR&ll=40.416865400000006,-3.7043021694877964&v=20210101&query=Italian&radius=3000&limit=30'

In [5]:
results = requests.get(url).json()

In [6]:
venues = results['response']['venues']
dataframe = pd.json_normalize(venues)
dataframe.shape

(27, 18)

In [7]:
#dataframe

In [8]:
#solamente dejamos las columnas name, categories, id y todas las que comienzan con "location"
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
filtered_columns

['name',
 'categories',
 'location.address',
 'location.lat',
 'location.lng',
 'location.labeledLatLngs',
 'location.distance',
 'location.postalCode',
 'location.cc',
 'location.city',
 'location.state',
 'location.country',
 'location.formattedAddress',
 'location.crossStreet',
 'id']

In [9]:
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered.head()

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,id
0,iS Italian Skills,"[{'id': '4bf58dd8d48988d104951735', 'name': 'B...","C. Gravina, 9",40.423069,-3.697672,"[{'label': 'display', 'lat': 40.42306918055597...",890,28004,ES,Madrid,Madrid,España,"[C. Gravina, 9, 28004 Madrid Madrid, España]",NaN,52ed42bc11d28180d18d8944
1,italiana_madrid,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...","Corredera Baja de San Pablo,10",40.422139,-3.704285,"[{'label': 'display', 'lat': 40.42213863996650...",587,28004,ES,Madrid,Madrid,España,"[Corredera Baja de San Pablo,10, 28004 Madrid ...",NaN,509d4c9ee4b05e06b4409edd
2,Embajada de la República Italiana - Ambasciata...,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...","Calle de Lagasca, 98",40.432600,-3.684364,"[{'label': 'display', 'lat': 40.43260015265081...",2433,28006,ES,Madrid,Madrid,España,"[Calle de Lagasca, 98, 28006 Madrid Madrid, Es...",NaN,4c0958e3ffb8c9b6bdc26961
3,Molto Italiano,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,40.415978,-3.705748,"[{'label': 'display', 'lat': 40.415978, 'lng':...",157,NaN,ES,NaN,NaN,España,[España],NaN,4e5e767c14954da39ff06d12
4,Italianos en Linneo,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...","Calle de Linneo, 40, 28005 Madrid",40.409846,-3.720492,"[{'label': 'display', 'lat': 40.40984599254621...",1579,28005,ES,Madrid,Madrid,España,"[Calle de Linneo, 40, 28005 Madrid, 28005 Madr...",NaN,4f0225215c5c51dd308b55d7


In [10]:
# funcion que extrae la categoria del venue
def get_category_type(x):
    try:
        categories_list = x['categories']    
    except:
        categories_list = x['venue.categories']  
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [11]:
# filtramos la categoria de cada fila
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.head()

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,id
0,iS Italian Skills,Boutique,"C. Gravina, 9",40.423069,-3.697672,"[{'label': 'display', 'lat': 40.42306918055597...",890,28004,ES,Madrid,Madrid,España,"[C. Gravina, 9, 28004 Madrid Madrid, España]",NaN,52ed42bc11d28180d18d8944
1,italiana_madrid,Café,"Corredera Baja de San Pablo,10",40.422139,-3.704285,"[{'label': 'display', 'lat': 40.42213863996650...",587,28004,ES,Madrid,Madrid,España,"[Corredera Baja de San Pablo,10, 28004 Madrid ...",NaN,509d4c9ee4b05e06b4409edd
2,Embajada de la República Italiana - Ambasciata...,Embassy / Consulate,"Calle de Lagasca, 98",40.432600,-3.684364,"[{'label': 'display', 'lat': 40.43260015265081...",2433,28006,ES,Madrid,Madrid,España,"[Calle de Lagasca, 98, 28006 Madrid Madrid, Es...",NaN,4c0958e3ffb8c9b6bdc26961
3,Molto Italiano,Pizza Place,NaN,40.415978,-3.705748,"[{'label': 'display', 'lat': 40.415978, 'lng':...",157,NaN,ES,NaN,NaN,España,[España],NaN,4e5e767c14954da39ff06d12
4,Italianos en Linneo,Bar,"Calle de Linneo, 40, 28005 Madrid",40.409846,-3.720492,"[{'label': 'display', 'lat': 40.40984599254621...",1579,28005,ES,Madrid,Madrid,España,"[Calle de Linneo, 40, 28005 Madrid, 28005 Madr...",NaN,4f0225215c5c51dd308b55d7


In [12]:
#Limpiamos los nombres de las columnas dejando solo el ultimo elemento
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,iS Italian Skills,Boutique,"C. Gravina, 9",40.423069,-3.697672,"[{'label': 'display', 'lat': 40.42306918055597...",890,28004,ES,Madrid,Madrid,España,"[C. Gravina, 9, 28004 Madrid Madrid, España]",NaN,52ed42bc11d28180d18d8944
1,italiana_madrid,Café,"Corredera Baja de San Pablo,10",40.422139,-3.704285,"[{'label': 'display', 'lat': 40.42213863996650...",587,28004,ES,Madrid,Madrid,España,"[Corredera Baja de San Pablo,10, 28004 Madrid ...",NaN,509d4c9ee4b05e06b4409edd
2,Embajada de la República Italiana - Ambasciata...,Embassy / Consulate,"Calle de Lagasca, 98",40.432600,-3.684364,"[{'label': 'display', 'lat': 40.43260015265081...",2433,28006,ES,Madrid,Madrid,España,"[Calle de Lagasca, 98, 28006 Madrid Madrid, Es...",NaN,4c0958e3ffb8c9b6bdc26961
3,Molto Italiano,Pizza Place,NaN,40.415978,-3.705748,"[{'label': 'display', 'lat': 40.415978, 'lng':...",157,NaN,ES,NaN,NaN,España,[España],NaN,4e5e767c14954da39ff06d12
4,Italianos en Linneo,Bar,"Calle de Linneo, 40, 28005 Madrid",40.409846,-3.720492,"[{'label': 'display', 'lat': 40.40984599254621...",1579,28005,ES,Madrid,Madrid,España,"[Calle de Linneo, 40, 28005 Madrid, 28005 Madr...",NaN,4f0225215c5c51dd308b55d7


In [13]:
df = dataframe_filtered[["id","name","address","categories","postalCode","lat","lng"]]
df.head()

,id,name,address,categories,postalCode,lat,lng
0,52ed42bc11d28180d18d8944,iS Italian Skills,"C. Gravina, 9",Boutique,28004,40.423069,-3.697672
1,509d4c9ee4b05e06b4409edd,italiana_madrid,"Corredera Baja de San Pablo,10",Café,28004,40.422139,-3.704285
2,4c0958e3ffb8c9b6bdc26961,Embajada de la República Italiana - Ambasciata...,"Calle de Lagasca, 98",Embassy / Consulate,28006,40.432600,-3.684364
3,4e5e767c14954da39ff06d12,Molto Italiano,NaN,Pizza Place,NaN,40.415978,-3.705748
4,4f0225215c5c51dd308b55d7,Italianos en Linneo,"Calle de Linneo, 40, 28005 Madrid",Bar,28005,40.409846,-3.720492


In [14]:
df

,id,name,address,categories,postalCode,lat,lng
0,52ed42bc11d28180d18d8944,iS Italian Skills,"C. Gravina, 9",Boutique,28004,40.423069,-3.697672
1,509d4c9ee4b05e06b4409edd,italiana_madrid,"Corredera Baja de San Pablo,10",Café,28004,40.422139,-3.704285
2,4c0958e3ffb8c9b6bdc26961,Embajada de la República Italiana - Ambasciata...,"Calle de Lagasca, 98",Embassy / Consulate,28006,40.432600,-3.684364
3,4e5e767c14954da39ff06d12,Molto Italiano,NaN,Pizza Place,NaN,40.415978,-3.705748
4,4f0225215c5c51dd308b55d7,Italianos en Linneo,"Calle de Linneo, 40, 28005 Madrid",Bar,28005,40.409846,-3.720492
5,4d96e9e4b188721e66580237,Librería Caffe Italiana Madrid,"Corredera baja de San Pablo, 10",Café,28004,40.422134,-3.704275
6,4c080ac3a1b32d7f3c6595f0,4D Caffetteria Italiana,"C/ de Cuchilleros, 17",Café,28005,40.413972,-3.708090
7,5390ee01498e87f1a952ff0e,Bellamia Gelateria Italiana,"C. Pérez Galdós, 3",Ice Cream Shop,28004,40.422448,-3.700408
8,4d207214756e8cfa645a6c54,Pizzeria Italiana Piccolo Diavolo,"C. Fúcar, 11",Italian Restaurant,28014,40.411529,-3.695596
9,5b56485ba42362002c87a1f5,La Capricciosa Italiana Pizzeria,Calle de las Maldonadas,Pizza Place,28005,40.410920,-3.707741


In [15]:
rest = []
for i in df.categories.unique():
    if (i.find("Rest")!=-1) or (i.find("Pizza")!=-1)or (i.find("Café")!=-1):
        rest.append(i)
rest

['Café', 'Pizza Place', 'Italian Restaurant']

In [16]:
df = df[df.categories.isin(rest)]
df.head()

,id,name,address,categories,postalCode,lat,lng
1,509d4c9ee4b05e06b4409edd,italiana_madrid,"Corredera Baja de San Pablo,10",Café,28004,40.422139,-3.704285
3,4e5e767c14954da39ff06d12,Molto Italiano,NaN,Pizza Place,NaN,40.415978,-3.705748
5,4d96e9e4b188721e66580237,Librería Caffe Italiana Madrid,"Corredera baja de San Pablo, 10",Café,28004,40.422134,-3.704275
6,4c080ac3a1b32d7f3c6595f0,4D Caffetteria Italiana,"C/ de Cuchilleros, 17",Café,28005,40.413972,-3.708090
8,4d207214756e8cfa645a6c54,Pizzeria Italiana Piccolo Diavolo,"C. Fúcar, 11",Italian Restaurant,28014,40.411529,-3.695596


In [17]:
df

,id,name,address,categories,postalCode,lat,lng
1,509d4c9ee4b05e06b4409edd,italiana_madrid,"Corredera Baja de San Pablo,10",Café,28004,40.422139,-3.704285
3,4e5e767c14954da39ff06d12,Molto Italiano,NaN,Pizza Place,NaN,40.415978,-3.705748
5,4d96e9e4b188721e66580237,Librería Caffe Italiana Madrid,"Corredera baja de San Pablo, 10",Café,28004,40.422134,-3.704275
6,4c080ac3a1b32d7f3c6595f0,4D Caffetteria Italiana,"C/ de Cuchilleros, 17",Café,28005,40.413972,-3.708090
8,4d207214756e8cfa645a6c54,Pizzeria Italiana Piccolo Diavolo,"C. Fúcar, 11",Italian Restaurant,28014,40.411529,-3.695596
9,5b56485ba42362002c87a1f5,La Capricciosa Italiana Pizzeria,Calle de las Maldonadas,Pizza Place,28005,40.410920,-3.707741
11,4ca8e61e9f4776b0150d9a13,Pizzeria Italiana Mamma Mia,"Costanilla Capuchinos, 3",Pizza Place,28004,40.421416,-3.699332
15,4cc87b3dd147a093179ce1ae,Aperitivo Italiano en Malasaña,Calle Apodaca 3,Italian Restaurant,28004,40.427283,-3.701199
17,56cee7cecd10c1fb90cbd632,Cucina italiana,NaN,Italian Restaurant,NaN,40.426106,-3.688721
19,532d7469498e6f7a11b53ddc,Matteo Cucina Italiana,Calle Ayala 28,Italian Restaurant,28001,40.427054,-3.685701


3.	En folium úbica los restaurantes.


In [18]:
mapa = folium.Map(location=[latitude, longitude], zoom_start=13) 

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Centro Madrid',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(mapa)

# los circulos azules son los restaurantes
for lat, lng, label in zip(df.lat, df.lng, df.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(mapa)

mapa

4.	Obtén la siguiente información de los primeros 3 restaurantes italianos: name, rating, likes y dislike y location, ordenarlos en un DataFrame. 


In [19]:
df.head(3).id

1    509d4c9ee4b05e06b4409edd
3    4e5e767c14954da39ff06d12
5    4d96e9e4b188721e66580237
Name: id, dtype: object

In [20]:
venue_id = '4e5e767c14954da39ff06d12' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4e5e767c14954da39ff06d12?client_id=U1LHGA0A24XNVNYN4FQJK3MD1VCBZ0KPP2LO12XQTS3FCN21&client_secret=N5HHZVFVPRVU2A3BMHFXU2UW1ODJ5QIM4PUQM0UJSCFIYRNR&v=20210101'

In [21]:
result = requests.get(url).json()
venue = result['response']['venue']

In [22]:
venue.keys()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])

In [23]:
def get_info(venue_id):

    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    venue = result['response']['venue']

    nombre = venue["name"]
    ubic = venue["location"]["country"]
    likes = venue["likes"]["count"]
    dislikes = venue["dislike"]
    try: 
        rating = venue["rating"]
    except:
        rating = False
    
    return [venue_id, nombre, ubic, likes, dislikes, rating]

In [24]:
lista = []
for i in df.head(3).id:
    lista.append(get_info(i))

In [25]:
lista

[['509d4c9ee4b05e06b4409edd', 'italiana_madrid', 'España', 6, False, 7.3],
 ['4e5e767c14954da39ff06d12', 'Molto Italiano', 'España', 0, False, False],
 ['4d96e9e4b188721e66580237',
  'Librería Caffe Italiana Madrid',
  'España',
  109,
  False,
  8.0]]

In [26]:
df1 = pd.DataFrame(lista, columns=["id","nombre","pais","likes","dislikes","rating"] )

In [27]:
df1

,id,nombre,pais,likes,dislikes,rating
0,509d4c9ee4b05e06b4409edd,italiana_madrid,España,6,False,7.3
1,4e5e767c14954da39ff06d12,Molto Italiano,España,0,False,False
2,4d96e9e4b188721e66580237,Librería Caffe Italiana Madrid,España,109,False,8


5.	Con la misma dirección obtén los primeros 20 venues más cercanos y haz un filtrado por la categoría que más se presenta y guardarlos en un dataframe. (Por ejemplo, si lo que más hay son Gimnasios, guardar esos gimnasios en un dataframe)


In [28]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
venues = results["response"]["groups"][0]['items']
len(venues)

30

In [29]:
df = pd.json_normalize(venues) # pasamos el JSON a dataframe

filtered_columns = ['venue.name', 'venue.categories'] + [col for col in df.columns if col.startswith('venue.location.')] + ['venue.id']
df1 = df.loc[:, filtered_columns]

df1['venue.categories'] = df1.apply(get_category_type, axis=1)
df1

,venue.name,venue.categories,venue.location.address,venue.location.lat,venue.location.lng,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.crossStreet,venue.location.neighborhood,venue.id
0,Puerta del Sol,Plaza,Pl. Puerta del Sol,40.417034,-3.705251,82,28013,ES,Madrid,Madrid,España,"[Pl. Puerta del Sol, 28013 Madrid Madrid, España]",NaN,NaN,NaN,4adcda37f964a5201f3c21e3
1,Club del Gourmet Corte Ingles,Gourmet Shop,"C. Preciados, 3",40.417497,-3.704686,77,28013,ES,Madrid,Madrid,España,"[C. Preciados, 3, 28013 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.41749665225322...",NaN,NaN,4bddaac30ee3a59370242eb0
2,LUSH,Cosmetics Shop,Calle del Carmen 24,40.419012,-3.704898,244,28013,ES,Madrid,Madrid,España,"[Calle del Carmen 24, 28013 Madrid Madrid, Esp...","[{'label': 'display', 'lat': 40.41901214762635...",NaN,NaN,5431272e498e8e357f694549
3,SunGate Hostel,Hostel,"Calle del Carmen, 16 - 2º Izq",40.418696,-3.704623,205,28013,ES,Madrid,Madrid,España,"[Calle del Carmen, 16 - 2º Izq, 28013 Madrid M...","[{'label': 'display', 'lat': 40.41869565648317...",NaN,NaN,5476a1ec498eb0f50cb79122
4,Hotel Liabeny,Hotel,"C. Salud, 3",40.418742,-3.703996,210,28013,ES,Madrid,Madrid,España,"[C. Salud, 3, 28013 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.41874245617109...",NaN,NaN,4adcda33f964a520aa3a21e3
5,Plaza Mayor,Plaza,Pl. Mayor,40.415527,-3.707506,309,28012,ES,Madrid,Madrid,España,"[Pl. Mayor, 28012 Madrid Madrid, España]",NaN,NaN,NaN,4adcda37f964a520193c21e3
6,Rosi La Loca,Tapas Restaurant,Calle Cadiz 4,40.415821,-3.702955,162,28012,ES,Madrid,Madrid,España,"[Calle Cadiz 4, 28012 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.41582101171752...",NaN,NaN,5478dde0498eab68d4d6423f
7,La Pulpería de Victoria,Seafood Restaurant,"Calle de La Victoria, 2, (Junto a Puerta del Sol)",40.416506,-3.701709,223,28012,ES,Madrid,Madrid,España,"[Calle de La Victoria, 2, (Junto a Puerta del ...","[{'label': 'display', 'lat': 40.416506381, 'ln...",Carrera de San Jerónimo,NaN,4f33cbace4b0b5f6dee6d145
8,Apple Puerta del Sol,Electronics Store,"Plaza De La Puerta Del Sol, 1",40.416900,-3.702251,173,28013,ES,Madrid,Madrid,España,"[Plaza De La Puerta Del Sol, 1 (C. Alcalá), 28...","[{'label': 'display', 'lat': 40.4169, 'lng': -...",C. Alcalá,NaN,538a4c78498e264f18c776b2
9,Plaza de Santa Ana,Plaza,Pl. de Santa Ana,40.414631,-3.701033,372,28012,ES,Madrid,Madrid,España,"[Pl. de Santa Ana, 28012 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.41463076310542...",NaN,NaN,4b72841ff964a520db7c2de3


In [30]:
# limpiamos los nombres de columnas
df1.columns = [col.split('.')[-1] for col in df1.columns]
df1.head(10)

,name,categories,address,lat,lng,distance,postalCode,cc,city,state,country,formattedAddress,labeledLatLngs,crossStreet,neighborhood,id
0,Puerta del Sol,Plaza,Pl. Puerta del Sol,40.417034,-3.705251,82,28013,ES,Madrid,Madrid,España,"[Pl. Puerta del Sol, 28013 Madrid Madrid, España]",NaN,NaN,NaN,4adcda37f964a5201f3c21e3
1,Club del Gourmet Corte Ingles,Gourmet Shop,"C. Preciados, 3",40.417497,-3.704686,77,28013,ES,Madrid,Madrid,España,"[C. Preciados, 3, 28013 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.41749665225322...",NaN,NaN,4bddaac30ee3a59370242eb0
2,LUSH,Cosmetics Shop,Calle del Carmen 24,40.419012,-3.704898,244,28013,ES,Madrid,Madrid,España,"[Calle del Carmen 24, 28013 Madrid Madrid, Esp...","[{'label': 'display', 'lat': 40.41901214762635...",NaN,NaN,5431272e498e8e357f694549
3,SunGate Hostel,Hostel,"Calle del Carmen, 16 - 2º Izq",40.418696,-3.704623,205,28013,ES,Madrid,Madrid,España,"[Calle del Carmen, 16 - 2º Izq, 28013 Madrid M...","[{'label': 'display', 'lat': 40.41869565648317...",NaN,NaN,5476a1ec498eb0f50cb79122
4,Hotel Liabeny,Hotel,"C. Salud, 3",40.418742,-3.703996,210,28013,ES,Madrid,Madrid,España,"[C. Salud, 3, 28013 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.41874245617109...",NaN,NaN,4adcda33f964a520aa3a21e3
5,Plaza Mayor,Plaza,Pl. Mayor,40.415527,-3.707506,309,28012,ES,Madrid,Madrid,España,"[Pl. Mayor, 28012 Madrid Madrid, España]",NaN,NaN,NaN,4adcda37f964a520193c21e3
6,Rosi La Loca,Tapas Restaurant,Calle Cadiz 4,40.415821,-3.702955,162,28012,ES,Madrid,Madrid,España,"[Calle Cadiz 4, 28012 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.41582101171752...",NaN,NaN,5478dde0498eab68d4d6423f
7,La Pulpería de Victoria,Seafood Restaurant,"Calle de La Victoria, 2, (Junto a Puerta del Sol)",40.416506,-3.701709,223,28012,ES,Madrid,Madrid,España,"[Calle de La Victoria, 2, (Junto a Puerta del ...","[{'label': 'display', 'lat': 40.416506381, 'ln...",Carrera de San Jerónimo,NaN,4f33cbace4b0b5f6dee6d145
8,Apple Puerta del Sol,Electronics Store,"Plaza De La Puerta Del Sol, 1",40.416900,-3.702251,173,28013,ES,Madrid,Madrid,España,"[Plaza De La Puerta Del Sol, 1 (C. Alcalá), 28...","[{'label': 'display', 'lat': 40.4169, 'lng': -...",C. Alcalá,NaN,538a4c78498e264f18c776b2
9,Plaza de Santa Ana,Plaza,Pl. de Santa Ana,40.414631,-3.701033,372,28012,ES,Madrid,Madrid,España,"[Pl. de Santa Ana, 28012 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.41463076310542...",NaN,NaN,4b72841ff964a520db7c2de3


In [31]:
df1.categories.value_counts()

Plaza                     6
Hotel                     5
Gourmet Shop              2
Tapas Restaurant          2
Spanish Restaurant        1
Clothing Store            1
Market                    1
History Museum            1
Other Nightlife           1
Theater                   1
Argentinian Restaurant    1
Cocktail Bar              1
Cosmetics Shop            1
Seafood Restaurant        1
Mexican Restaurant        1
Hostel                    1
Restaurant                1
Pastry Shop               1
Electronics Store         1
Name: categories, dtype: int64

In [32]:
df = df1[df1.categories=="Hotel"]
df

,name,categories,address,lat,lng,distance,postalCode,cc,city,state,country,formattedAddress,labeledLatLngs,crossStreet,neighborhood,id
4,Hotel Liabeny,Hotel,"C. Salud, 3",40.418742,-3.703996,210,28013,ES,Madrid,Madrid,España,"[C. Salud, 3, 28013 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.41874245617109...",NaN,NaN,4adcda33f964a520aa3a21e3
12,The Hat Madrid,Hotel,Imperial 9,40.414343,-3.707120,368,28012,ES,Madrid,Madrid,España,"[Imperial 9, 28012 Madrid Madrid, España]","[{'label': 'display', 'lat': 40.41434279603091...",NaN,NaN,53106df211d2057452905bb1
19,Hotel NH Collection Madrid Palacio de Tepa,Hotel,"C. San Sebastián, 2",40.413800,-3.701676,407,28012,ES,Madrid,Madrid,España,"[C. San Sebastián, 2, 28012 Madrid Madrid, Esp...","[{'label': 'display', 'lat': 40.41379972837331...",NaN,NaN,4cdd9af2df986ea876a5d616
25,Gran Meliá Palacio de los Duques *****,Hotel,"Cuesta de Santo Domingo, 5",40.419835,-3.709494,550,28013,ES,Madrid,Madrid,España,"[Cuesta de Santo Domingo, 5, 28013 Madrid Madr...","[{'label': 'display', 'lat': 40.41983538326362...",NaN,NaN,4adcda33f964a520a53a21e3
26,Axel Hotel Madrid,Hotel,"Calle de Atocha, 49",40.413181,-3.700569,517,28012,ES,Madrid,Madrid,España,"[Calle de Atocha, 49, 28012 Madrid Madrid, Esp...","[{'label': 'display', 'lat': 40.41318142339538...",NaN,NaN,599b767ef96b2c3bf7fcda3f


6.	Del dataframe anterior obtén name, rating y likes y selecciona al que tiene más rating y más likes. 


In [33]:
lista = []
for i in df.id:
    lista.append(get_info(i))

In [34]:
df1 = pd.DataFrame(lista, columns=["id","nombre","pais","likes","dislikes","rating"] )
df1

,id,nombre,pais,likes,dislikes,rating
0,4adcda33f964a520aa3a21e3,Hotel Liabeny,España,109,False,8.8
1,53106df211d2057452905bb1,The Hat Madrid,España,290,False,9.0
2,4cdd9af2df986ea876a5d616,Hotel NH Collection Madrid Palacio de Tepa,España,63,False,8.9
3,4adcda33f964a520a53a21e3,Gran Meliá Palacio de los Duques *****,España,67,False,9.0
4,599b767ef96b2c3bf7fcda3f,Axel Hotel Madrid,España,19,False,8.9


In [35]:
df1.sort_values("likes", ascending=False).head(1)

,id,nombre,pais,likes,dislikes,rating
1,53106df211d2057452905bb1,The Hat Madrid,España,290,False,9.0


In [36]:
df1.sort_values("rating", ascending=False).head(1)

,id,nombre,pais,likes,dislikes,rating
1,53106df211d2057452905bb1,The Hat Madrid,España,290,False,9.0


7.	En un Mapa muestra los venues anteriores. (solamente los venues que comparten la misma categoría mayoritaria)


In [37]:
mapa = folium.Map(location=[latitude, longitude], zoom_start=13) 


# los circulos azules son los restaurantes
for lat, lng, label in zip(df.lat, df.lng, df.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(mapa)

mapa